In [2]:
import numba
import numpy as np
from functools import reduce
from operator import mul

from minitorch.autodiff import tensor_functions as tf
from minitorch.autodiff.tensor_functions import SimpleOps, FastOps, TensorBackend
from minitorch import testing

In [3]:
FastBackend = TensorBackend(FastOps)
SimpleBackend = TensorBackend(SimpleOps)

In [11]:
t_simple = tf.rand(shape=(3, 2), backend=SimpleBackend)
t_fast = tf.rand(shape=(3, 2), backend=FastBackend)

In [6]:
t_simple


[
	[0.23923 0.57078]
	[0.38662 0.56879]
	[0.40609 0.99402]
]

In [7]:
t_simple + 6


[
	[6.23923 6.57078]
	[6.38662 6.56879]
	[6.40609 6.99402]
]

In [12]:
t_fast


[
	[0.20644 0.86625]
	[0.49445 0.87160]
	[0.17285 0.64723]
]

In [13]:
t_fast + 6


[
	[6.20644 6.86625]
	[6.49445 6.87160]
	[6.17285 6.64723]
]